In [2]:
from tensorflow import keras
import numpy as np
import pandas as pd
import random
import os

dataRep = '../data/'
scriptRep = '../kuzushiji_recognition/'

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, scriptRep)
import progressBar

In [7]:
testFrac = 0.15
valFrac = 0.15

unicodeData = pd.read_csv(dataRep+'unicode_translation.csv')

raw = np.load('../data/dataset/caracterClassificationFull.npz')
image = np.expand_dims(raw['image'], 3).copy()
charOutput = raw['characterClass'].copy()
del raw

index = np.arange(image.shape[0])
np.random.shuffle(index)
image = image[index]/255.0
charOutput = charOutput[index]
del index

nFrac = int(image.shape[0]*(1.-testFrac))
nVal = int(image.shape[0]*(1.-testFrac-valFrac))
print(image.shape[0], image.shape[1:], nVal, nFrac)

500 (32, 32, 1) 350 425


In [8]:
def trainAndEvaluateModel(model, filenameModel, batchSize):
    
    checkpoint = keras.callbacks.ModelCheckpoint(filepath=filenameModel,
                                                 monitor='val_loss',
                                                 verbose=0,
                                                 save_best_only=True,
                                                 mode='auto', period=1)
    history = keras.callbacks.History()
    reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 factor=0.1, patience=2,
                                                 verbose=0,
                                                 mode='auto')
    earlyStop = keras.callbacks.EarlyStopping(monitor='val_loss',
                                              min_delta=1e-7, patience=5,
                                              verbose=0, mode='auto')
    callbacks = [checkpoint, history, reduceLR, earlyStop]
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(image[:nVal], charOutput[:nVal], epochs=50, batch_size=batchSize,
              validation_data=(image[nVal:nFrac], charOutput[nVal:nFrac]), shuffle=True, callbacks=callbacks)
    
    model = keras.models.load_model(filenameModel)
    test_loss, test_acc = model.evaluate(image[nVal:nFrac], charOutput[nVal:nFrac])
    
    return test_loss


def optimizeHyperParameter(funcCreateModel, filenameModel, dropoutLimit=(0,20), batchSizeLimit=(0,10), convolutionLayerLimit=(2,10), denseLayerLimit=(2,10), maxIter=30):
    
    i=0
    stop=False
    curentParam = [6, 5, 4, 8] # (dropout, batchsize, convolution, dense)
    testedParam = np.zeros((dropoutLimit[1]+1, batchSizeLimit[1]+1, convolutionLayerLimit[1]+1, denseLayerLimit[1]+1), dtype=np.bool)
    
    stdLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0]/20., 2**(curentParam[2]), 2**(curentParam[3])), filenameModel, 2**(curentParam[1]))
    testedParam[curentParam[0], curentParam[1], curentParam[2], curentParam[3]] = True
    
    while (i<maxIter) and not stop:
        i+=1
        stop=True
        
        if (curentParam[0]-1) >= dropoutLimit[0] and not testedParam[curentParam[0]-1, curentParam[1], curentParam[2], curentParam[3]]:
            print("Test dropout down")
            testLoss = trainAndEvaluateModel(funcCreateModel(float(curentParam[0]-1)/20., 2**(curentParam[2]), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]))
            testedParam[curentParam[0]-1, curentParam[1], curentParam[2], curentParam[3]] = True
            
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[0] -= 1
                os.system("cp temp.h5 "+filenameModel)
                print("New param set", curentParam)
            
        if (curentParam[0]+1) <= dropoutLimit[1] and not testedParam[curentParam[0]+1, curentParam[1], curentParam[2], curentParam[3]]:
            print("Test dropout up")
            testLoss = trainAndEvaluateModel(funcCreateModel(float(curentParam[0]+1)/20., 2**(curentParam[2]), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]))
            testedParam[curentParam[0]+1, curentParam[1], curentParam[2], curentParam[3]] = True
            
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[0] += 1
                os.system("cp temp.h5 "+filenameModel)
                print("New param set", curentParam)
                
                
        if (curentParam[1]-1) >= batchSizeLimit[0] and not testedParam[curentParam[0], curentParam[1]-1, curentParam[2], curentParam[3]]:
            print("Test batchsize down")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0]/20., 2**(curentParam[2]), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]-1))
            testedParam[curentParam[0], curentParam[1]-1, curentParam[2], curentParam[3]] = True
            
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[1] -= 1
                os.system("cp temp.h5 "+filenameModel)
                print("New param set", curentParam)
            
        if (curentParam[1]+1) <= batchSizeLimit[1] and not testedParam[curentParam[0], curentParam[1]+1, curentParam[2], curentParam[3]]:
            print("Test batchsize up")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0]/20., 2**(curentParam[2]), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]+1))
            testedParam[curentParam[0], curentParam[1]+1, curentParam[2], curentParam[3]] = True
            
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[1] += 1
                os.system("cp temp.h5 "+filenameModel)
                print("New param set", curentParam)
                
        if (curentParam[2]-1) >= convolutionLayerLimit[0] and not testedParam[curentParam[0], curentParam[1], curentParam[2]-1, curentParam[3]]:
            print("Test convolution down")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0]/20., 2**(curentParam[2]-1), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]))
            testedParam[curentParam[0], curentParam[1], curentParam[2]-1, curentParam[3]] = True
            
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[2] -= 1
                os.system("cp temp.h5 "+filenameModel)
                print("New param set", curentParam)
            
        if (curentParam[2]+1) <= convolutionLayerLimit[1] and not testedParam[curentParam[0]-1, curentParam[1], curentParam[2]+1, curentParam[3]]:
            print("Test convolution up")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0]/20., 2**(curentParam[2]+1), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]))
            testedParam[curentParam[0]-1, curentParam[1], curentParam[2]+1, curentParam[3]] = True
            
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[2] += 1
                os.system("cp temp.h5 "+filenameModel)
                print("New param set", curentParam)
                
        if (curentParam[3]-1) >= denseLayerLimit[0] and not testedParam[curentParam[0]-1, curentParam[1], curentParam[2], curentParam[3]-1]:
            print("Test dense down")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0]/20., 2**(curentParam[2]), 2**(curentParam[3]-1)),
                                             'temp.h5', 2**(curentParam[1]))
            testedParam[curentParam[0]-1, curentParam[1], curentParam[2], curentParam[3]-1] = True
            
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[3] -= 1
                os.system("cp temp.h5 "+filenameModel)
                print("New param set", curentParam)
            
        if (curentParam[3]+1) <= denseLayerLimit[1] and not testedParam[curentParam[0]-1, curentParam[1], curentParam[2], curentParam[3]+1]:
            print("Test dense up")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0]/20., 2**(curentParam[2]), 2**(curentParam[3]+1)),
                                             'temp.h5', 2**(curentParam[1]))
            testedParam[curentParam[0]-1, curentParam[1], curentParam[2], curentParam[3]+1] = True
            
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[3] += 1
                os.system("cp temp.h5 "+filenameModel)
                print("New param set", curentParam)
                
    print('Dropout :', curentParam[0])
    print('Batch Size :', 2**curentParam[1])
    print('Convolutional layer :', 2**curentParam[2])
    print('Dense Layer :', 2**curentParam[3])
    os.system("rm temp.h5")
    
    print("Nb iteration", i, "/", maxIter)
    print("Nb case tested", np.sum(testedParam), "/", np.sum(np.ones(testedParam.shape, dtype=np.bool)))
        
def createModel1(dropoutRate, convLayer, denseLayer):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(convLayer, (3, 3), activation='relu', input_shape=image.shape[1:]))
    model.add(keras.layers.SpatialDropout2D(dropoutRate))
    model.add(keras.layers.Conv2D(convLayer, (3, 3), activation='relu'))
    model.add(keras.layers.SpatialDropout2D(dropoutRate))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(convLayer*2, (3, 3), activation='relu'))
    model.add(keras.layers.SpatialDropout2D(dropoutRate))
    model.add(keras.layers.Conv2D(convLayer*2, (3, 3), activation='relu'))
    model.add(keras.layers.SpatialDropout2D(dropoutRate))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(denseLayer, activation='relu'))
    model.add(keras.layers.Dropout(dropoutRate))
    model.add(keras.layers.Dense(len(unicodeData), activation='softmax'))
    
    return model
    

In [9]:
optimizeHyperParameter(createModel1, '../models/KMNIST1.h5')

W0904 13:43:56.290018 140075983804224 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 350 samples, validate on 75 samples
Epoch 1/50
350/350 [==============================] - 2s 6ms/sample - loss: 8.0867 - acc: 0.0171 - val_loss: 6.1912 - val_acc: 0.0533
Epoch 2/50
350/350 [==============================] - 0s 1ms/sample - loss: 5.9530 - acc: 0.0257 - val_loss: 5.9086 - val_acc: 0.0133
Epoch 3/50
350/350 [==============================] - 0s 1ms/sample - loss: 5.2154 - acc: 0.0200 - val_loss: 5.8171 - val_acc: 0.0533
Epoch 4/50
350/350 [==============================] - 0s 1ms/sample - loss: 4.9712 - acc: 0.0286 - val_loss: 5.6959 - val_acc: 0.0133
Epoch 5/50
350/350 [==============================] - 0s 857us/sample - loss: 4.7841 - acc: 0.0371 - val_loss: 5.7446 - val_acc: 0.0533
Epoch 6/50
350/350 [==============================] - 0s 1ms/sample - loss: 4.7696 - acc: 0.0314 - val_loss: 5.7801 - val_acc: 0.0533
Epoch 7/50
350/350 [==============================] - 1s 3ms/sample - loss: 4.7084 - acc: 0.0314 - val_loss: 5.7772 - val_acc: 0.0533
Epoch 8/50
350/

W0904 13:44:04.486896 140075983804224 deprecation.py:506] From /home/mathieu/miniconda3/envs/mlearning/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0904 13:44:04.489238 140075983804224 deprecation.py:506] From /home/mathieu/miniconda3/envs/mlearning/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


75/75 [==============================] - 0s 2ms/sample - loss: 5.6959 - acc: 0.0133
Test dropout down


W0904 13:44:05.984537 140075983804224 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


...

W0904 13:55:36.837564 140075983804224 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 350 samples, validate on 75 samples
Epoch 1/50
350/350 [==============================] - 13s 37ms/sample - loss: 7.5187 - acc: 0.0229 - val_loss: 6.8161 - val_acc: 0.0533
Epoch 2/50
350/350 [==============================] - 3s 8ms/sample - loss: 6.1269 - acc: 0.0343 - val_loss: 5.8188 - val_acc: 0.0533
Epoch 3/50
350/350 [==============================] - 5s 14ms/sample - loss: 5.2668 - acc: 0.0429 - val_loss: 5.5933 - val_acc: 0.0533
Epoch 4/50
350/350 [==============================] - 4s 12ms/sample - loss: 4.9524 - acc: 0.0343 - val_loss: 5.7043 - val_acc: 0.0533
Epoch 5/50
350/350 [==============================] - 7s 19ms/sample - loss: 4.8043 - acc: 0.0457 - val_loss: 5.8730 - val_acc: 0.0533
Epoch 6/50
350/350 [==============================] - 3s 7ms/sample - loss: 4.6895 - acc: 0.0400 - val_loss: 5.8469 - val_acc: 0.0533
Epoch 7/50
350/350 [==============================] - 4s 13ms/sample - loss: 4.7269 - acc: 0.0514 - val_loss: 5.8475 - val_acc: 0.0533
Epoch 8/50


W0904 13:56:35.821409 140075983804224 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Test batchsize down
Train on 350 samples, validate on 75 samples
Epoch 1/50
348/350 [============================>.] - ETA: 0s - loss: 7.3821 - acc: 0.0201

KeyboardInterrupt: 